In [1]:
import os
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
# or '2' to filter out INFO messages too
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
from config import update_settings, get_settings

update_settings('laptop', 'monkaa')
DATA_DIR, WEIGHTS_DIR, RESULTS_SAVE_DIR, LOG_DIR = get_settings()['dirs']
from data_utils import create_dataset_from_serialized_generator

# dataset = tf.data.Dataset.range(10)
# tempdataset = dataset.take(7).repeat()
# valdataset = dataset.skip(7).repeat()

# def get_test_batch():
#         try:
#             for batch in tempdataset:
#                 print(batch.numpy())
#         except StopIteration:
#             print("End of dataset")

# while True:
#       b = get_test_batch()
#       print(b)

args = {
    'data_subset': 'family_x2',
}

# Training data
assert os.path.exists(DATA_DIR + 'disparity/' + args['data_subset'] + '/left/'), "Improper data_subset selected"
pfm_paths = []
pfm_paths.append(DATA_DIR + 'disparity/' + args['data_subset'] + '/left/')
pfm_paths.append(DATA_DIR + 'material_index/' + args['data_subset'] + '/left/')
pfm_paths.append(DATA_DIR + 'object_index/' + args['data_subset'] + '/left/')
pfm_paths.append(DATA_DIR + 'optical_flow/' + args['data_subset'] + '/into_future/left/')
pgm_paths = []
pgm_paths.append(DATA_DIR + 'motion_boundaries/' + args['data_subset'] + '/into_future/left/')
png_paths = []
png_paths.append(DATA_DIR + 'frames_cleanpass/' + args['data_subset'] + '/left')
num_sources = len(pfm_paths) + len(pgm_paths) + len(png_paths)

# Training parameters
nt = 10  # number of time steps
nb_epoch = 150  # 150
batch_size = 4  # 4
sequences_per_epoch_train = 10  # 500
sequences_per_epoch_val = 1  # 500
assert sequences_per_epoch_train is None or type(sequences_per_epoch_train) == int # this will override the default of (dataset size / batch size)
assert sequences_per_epoch_val is None or type(sequences_per_epoch_val) == int # this will override the default of (dataset size / batch size)
# N_seq_val = 20  # number of sequences to use for validation
# num_P_CNN = args["num_P_CNN"]
# num_R_CLSTM = args["num_R_CLSTM"]
# output_channels = args["output_channels"]
original_im_shape = (540, 960, 3)
downscale_factor = 4
im_shape = (original_im_shape[0] // downscale_factor, original_im_shape[1] // downscale_factor, 3)

ts = .7
vs = (1 - ts) / 2
#  Create and split dataset
datasets, length = create_dataset_from_serialized_generator(pfm_paths, pgm_paths, png_paths, output_mode='Error',
                                                        im_height=im_shape[0], im_width=im_shape[1],
                                                        batch_size=batch_size, nt=nt, reserialize=False, shuffle=True, resize=True)

train_size = int(ts * length)
val_size = int(vs * length)
test_size = int(vs * length)
train_dataset, val_dataset, test_dataset = datasets
print(f"Train size: {train_size}")
print(f"Validation size: {val_size}")
print(f"Test size: {test_size}")
print("All datasets created successfully")

Using previously serialized dataset.
Begin tf.data.Dataset creation at 7.703399751335382e-05 seconds.
3 datasets created.
End tf.data.Dataset creation at 6.435531076996995 seconds.
Train size: 106
Validation size: 22
Test size: 22
All datasets created successfully


In [5]:

for batch in train_dataset:
    print(np.mean(batch[0][0]))

0, 38
1, 60
2, 94
3, 2
4, 97
45.413803
5, 87
6, 53
7, 34
8, 23
44.777298
9, 19
10, 83
11, 25
12, 50
45.29929
13, 22
14, 65
15, 90
16, 6
44.32845
17, 41
18, 72
19, 40
20, 80
45.02596
21, 102
22, 69
23, 35
24, 31
44.46981
25, 85
26, 52
27, 45
28, 71
44.56496
29, 100
30, 15
31, 18
32, 28
45.49008
33, 86
34, 95
35, 46
36, 81
44.97999
37, 98
38, 57
39, 0
40, 96
45.058414
41, 3
42, 48
43, 7
44, 63
46.048508
45, 58
46, 47
47, 93
48, 37
44.352093
49, 32
50, 74
51, 5
52, 89
45.201817
53, 88
54, 62
55, 14
56, 91
44.039745
57, 17
58, 42
59, 11
60, 9
45.234756
61, 66
62, 54
63, 30
64, 99
44.981094
65, 61
66, 13
67, 101
68, 64
45.6862
69, 76
70, 55
71, 84
72, 39
43.58017
73, 78
74, 21
75, 56
76, 44
44.705967
77, 73
78, 29
79, 79
80, 20
44.3121
81, 12
82, 27
83, 68
84, 26
45.367134
85, 75
86, 77
87, 33
88, 4
44.508453
89, 105
90, 51
91, 1
92, 82
45.722466
93, 43
94, 103
95, 70
96, 24
44.223816
97, 104
98, 10
99, 67
100, 36
45.122578
101, 92
102, 16
103, 59
104, 8
44.902424
105, 49
0, 75
45.623516
1,

KeyboardInterrupt: 

9, 7
10, 5
11, 56
12, 59
13, 48
14, 21
15, 39


: 

In [2]:
dit = iter(test_dataset)
# while True:
#     try:
#         batch = next(dit) # take just batch_x not batch_y
#     except StopIteration:
#         print("Stopped")
#         dit = iter(test_dataset)
#         batch = next(dit)
#     print(np.mean(batch[0][0]))
while True:
    batch = next(dit)    
    print(np.mean(batch[0][0]))

0, 136
1, 148
2, 145
3, 146
4, 142
46.423634
5, 147
6, 135
7, 143
8, 131
46.39489
9, 138
10, 144
11, 128
12, 141
46.144268
13, 140
14, 133
15, 139
16, 137
46.31312
17, 132
18, 130
19, 134
20, 129
46.08577
21, 149
0, 134
46.175507
1, 142
2, 135
3, 144
4, 133
46.308174
5, 146
6, 138
7, 149
8, 140
46.343872
9, 145
10, 143
11, 129
12, 139
46.299953
13, 136
14, 141
15, 137
16, 130
46.324444
17, 147
18, 131
19, 148
20, 128
46.214428
21, 132
0, 129
45.91709
1, 144
2, 128
3, 146
4, 131
46.159527
5, 137
6, 136
7, 133
8, 145
46.138866
9, 139
10, 141
11, 147
12, 143
46.43889
13, 130
14, 132
15, 148
16, 135
46.220963
17, 138
18, 142
19, 149
20, 140
46.360153
21, 134
0, 136
46.26209
1, 130
2, 128
3, 138
4, 145
46.071068
5, 135
6, 143
7, 144
8, 142
46.40054
9, 137
10, 139
11, 134
12, 149
46.304188
13, 148
14, 146
15, 147
16, 131
46.489254
17, 140
18, 129
19, 133
20, 132
46.07405
21, 141
0, 132
46.22068
1, 133
2, 135
3, 146
4, 130
46.191563
5, 136
6, 144
7, 139
8, 141
46.245453
9, 145
10, 143
11, 149

KeyboardInterrupt: 

13, 132
14, 136
15, 128
16, 141
17, 142
18, 137
19, 130


In [4]:

for batch in val_dataset:
    print(np.mean(batch[0][0]))

0, 127
1, 107
2, 110
3, 112
4, 118
45.61526
5, 120
6, 115
7, 119
8, 124
45.641224
9, 109
10, 111
11, 108
12, 125
45.65186
13, 114
14, 117
15, 113
16, 122
45.657486
17, 121
18, 123
19, 106
20, 126
45.499413
21, 116
0, 107
45.689205
1, 109
2, 112
3, 116
4, 113
45.582085
5, 127
6, 123
7, 124
8, 108
45.692146
9, 111
10, 118
11, 115
12, 125
45.64042
13, 106
14, 126
15, 120
16, 114
45.528275
17, 117
18, 121
19, 110
20, 122


KeyboardInterrupt: 

21, 119


In [3]:
for batch in test_dataset:
    print(np.mean(batch[0][0]))

0, 137
1, 138
2, 145
3, 132
4, 141
46.276554
5, 142
6, 140
7, 144
8, 128
46.433605
9, 143
10, 136
11, 149
12, 130
46.24987
13, 135
14, 148
15, 131
16, 139
46.137882
17, 146
18, 133
19, 134
20, 147
46.263626
21, 129
0, 128
46.175797
1, 147
2, 136
3, 141
4, 135
46.240093
5, 130
6, 140
7, 145
8, 139
46.243187
9, 149
10, 132
11, 137
12, 131
46.287685
13, 143
14, 133
15, 142
16, 134
46.236645
17, 144
18, 148
19, 146
20, 138
46.39559
21, 129
0, 134
46.092434
1, 139
2, 130
3, 131
4, 148
46.092224
5, 140
6, 129
7, 146
8, 142
46.30836
9, 128
10, 137
11, 135
12, 132


KeyboardInterrupt: 

13, 145
14, 149
15, 136
16, 144
17, 138
18, 133
19, 143


In [ ]:
def get_test_batch(dataset):
        try:
            for batch in dataset:
                print(np.mean(batch[0][0]))
        except StopIteration:
            print("End of dataset")

def get_test_batch2(dataset):
        dit = iter(dataset)
        try:
            batch = next(dit)
            print(np.mean(batch[0][0]))
        except StopIteration:
            print("End of dataset")
            dit = iter(dataset)
            batch = next(dit)
            print(np.mean(batch[0][0]))

while True:
      get_test_batch2(test_dataset)

In [2]:
import keras
import keras_cv
import tensorflow as tf

2024-05-16 23:00:14.254803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 23:00:14.254834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 23:00:14.255845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-16 23:00:14.261428: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 23:00:15.107546: W tensorflow/compiler/tf2

Using TensorFlow backend


/home/evalexii/miniconda3/envs/PPN/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
input = tf.ones((1, 5, 5, 16), dtype=tf.float32)
x = keras.layers.Conv2D(16, (3, 3))(input)
print(x.shape)
output = keras_cv.layers.SqueezeAndExcite2D(x.shape[-1])(x)
print(output.shape)

(1, 3, 3, 16)
(1, 3, 3, 16)


In [3]:
import tensorflow as tf

In [9]:
# Example tensor x with shape [batch, height, width, channels]
# Example: tf.random.normal([2, 5, 5, 3]) generates a tensor with 2 batches, 5x5 spatial dimensions, and 3 channels

shape = [2, 5, 5, 3]
single_channel_shape = shape[:-1] + [1]
x = tf.random.normal(shape)

# List of channels to set to zero (0-indexed)
channels_to_zero = [0, 2]  # Set channel 0 and channel 2 to zero

# Create a mask where selected channels are zero and others are one
channel_mask = tf.ones_like(x)
for channel in channels_to_zero:
    # Using tf.tensor_scatter_nd_update to set specific channels to zero in the mask
    index = tf.constant([[[[channel]]]])  # Note the four brackets for the index to match the tensor's 4D shape

    update = tf.zeros(single_channel_shape, dtype=x.dtype)
    channel_mask = tf.tensor_scatter_nd_update(channel_mask, index, update)

# Multiply the original tensor by the mask
result = x * channel_mask

print(result)

InvalidArgumentError: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} Outer dimensions of indices and update must match. Indices shape: [1,1,1,1], updates shape:[2,5,5,1] [Op:TensorScatterUpdate] name: 

In [13]:
# import tensorflow as tf

# Example tensor x with shape [batch, height, width, channels]
x = tf.random.normal([2, 5, 5, 4])  # Assume tensor with 4 channels

# List of channel indices to set to zero
channels_to_zero = [0, 1, 3]  # Zero out channels 0, 1, and 3

# Create a mask for the channels
mask = tf.constant([i not in channels_to_zero for i in range(x.shape[-1])], dtype=tf.float32)

# Reshape mask to be broadcastable across the batch and spatial dimensions
mask = tf.reshape(mask, [1, 1, 1, x.shape[-1]])

# Apply the mask to the tensor
x = x * mask

print(x[..., 2])


tf.Tensor(
[[[-1.4892974  -1.5584548  -0.40928242 -0.9602487   0.3120145 ]
  [ 1.0133559  -0.31203923 -0.19552547  0.53889555 -1.0778242 ]
  [-1.2669659   0.9875662  -1.7057128  -1.5694541  -0.8509229 ]
  [-0.7506281  -0.6145227   0.7555143   0.07049035  0.6204936 ]
  [-0.96874434 -0.53032476  0.02862556 -0.48450115 -0.4646648 ]]

 [[ 0.6145058  -0.14068246  0.6534147  -1.3314594  -1.580457  ]
  [-0.89542186  0.4455467   0.84875476 -0.58786786 -2.1768298 ]
  [ 0.22862211  1.0475172   0.7032318  -0.92729384  1.9831867 ]
  [ 1.4115286   1.2384869  -0.05710787 -1.1609427  -1.0961728 ]
  [-1.4623992   1.22778     2.1681988   0.22070688  0.8555502 ]]], shape=(2, 5, 5), dtype=float32)
